In [1]:
#Import Libraries

import os
import glob
import datetime

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import h5py
import json
import time

# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

Using TensorFlow backend.


In [2]:
config={
  "model"           : "vgg16",
  "weights"         : "imagenet",

  "train_path"      : "car_damage_dataset/training_severity",
  "test_path"       : "car_damage_dataset/validationv_severity",
  "features_path"   : "car_damage_check/severity/features.h5",
  "labels_path"     : "car_damage_check/severity/labels.h5",
  "classifier_path" : "car_damage_check/severity/classifier.pickle",
  "model_path"      : "car_damage_check/severity/model",

  "test_size"       : 0.20,
  "seed"            : 9,
}

In [3]:
# config variables
model_name    = config["model"]
weights     = config["weights"]
train_path    = config["train_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
model_path    = config["model_path"]

In [4]:
# Here we are loading the base VGG16 model with weights and then excluding the top dense layer
if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
else:
    base_model = None

print ("Successfully loaded base model and model...")

Successfully loaded base model and model...


/home/ec2-user/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("fc...)`


In [5]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [6]:
# path to training dataset
train_labels = os.listdir(train_path)

# encode the labels
print ("Encoding labels...")
le = LabelEncoder()
le.fit([tl for tl in train_labels])

# variables to hold features and labels
features = []
labels   = []

# loop over all the labels in the folder
count = 1
for i, label in enumerate(train_labels):
    cur_path = train_path + "/" + label
    count = 1
    for image_path in glob.glob(cur_path + "/*.jpg"):
        img = image.load_img(image_path, target_size=image_size)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
        labels.append(label)
        print ("Processed - " + str(count))
        count += 1
    print ("Completed label - " + label)

# encode the labels using LabelEncoder
le = LabelEncoder()
le_labels = le.fit_transform(labels)

# get the shape of training labels
print ("[STATUS] training labels: {}".format(le_labels))
print ("[STATUS] training labels shape: {}".format(le_labels.shape))

Encoding labels...
Processed - 1
Processed - 2
Processed - 3
Processed - 4
Processed - 5
Processed - 6
Processed - 7
Processed - 8
Processed - 9
Processed - 10
Processed - 11
Processed - 12
Processed - 13
Processed - 14
Processed - 15
Processed - 16
Processed - 17
Processed - 18
Processed - 19
Processed - 20
Processed - 21
Processed - 22
Processed - 23
Processed - 24
Processed - 25
Processed - 26
Processed - 27
Processed - 28
Processed - 29
Processed - 30
Processed - 31
Processed - 32
Processed - 33
Processed - 34
Processed - 35
Processed - 36
Processed - 37
Processed - 38
Processed - 39
Processed - 40
Processed - 41
Processed - 42
Processed - 43
Processed - 44
Processed - 45
Processed - 46
Processed - 47
Processed - 48
Processed - 49
Processed - 50
Processed - 51
Processed - 52
Processed - 53
Processed - 54
Processed - 55
Processed - 56
Processed - 57
Processed - 58
Processed - 59
Processed - 60
Processed - 61
Processed - 62
Processed - 63
Processed - 64
Processed - 65
Processed - 66


In [7]:
# save features and labels
h5f_data = h5py.File(features_path, 'w')
h5f_data.create_dataset('dataset_1', data=np.array(features))

h5f_label = h5py.File(labels_path, 'w')
h5f_label.create_dataset('dataset_1', data=np.array(le_labels))

h5f_data.close()
h5f_label.close()

# save model and weights
model_json = model.to_json()
with open(model_path + str(test_size) + ".json", "w") as json_file:
    json_file.write(model_json)

# save weights
model.save_weights(model_path + str(test_size) + ".h5")
print("Saved model and weights to disk..")

print ("Features and labels saved..")

# end time
end = time.time()
print ("End time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

Saved model and weights to disk..
Features and labels saved..
End time - 2020-12-17 00:14
